## AoC 2023 - Day xx

### Setup imports and get raw data

In [48]:
from collections import deque
#from icecream import ic
import pprint

SEARCH_MIN = 7
SEARCH_MAX = 27
data = open('test.txt').read().split('\n')

#SEARCH_MIN = 7
#SEARCH_MAX = 27
#data = open('input.txt').read().split('\n')


def line_intersection(line1, line2):
    xdiff = (line1[0][0] - line1[1][0], line2[0][0] - line2[1][0])
    ydiff = (line1[0][1] - line1[1][1], line2[0][1] - line2[1][1])

    def det(a, b):
        return a[0] * b[1] - a[1] * b[0]

    div = det(xdiff, ydiff)
    if div == 0:
        return False

    d = (det(*line1), det(*line2))
    x = det(d, xdiff) / div
    y = det(d, ydiff) / div
    return x, y

vectors = []
for line in data:
    vectors.append(tuple([int(thing) for half in line.split(" @ ") for thing in half.split(", ")]))


In [59]:
crossing_hailstones = set()
crossing_hailstones2 = set()
intersections = set()
#intersections = []
checked = set()

for thisv in vectors:
    for thatv in vectors:
        
        if (thatv,thisv) in checked or (thisv,thatv) in checked:
            print(f"already checked {(thatv,thisv)}")
            continue
        if thisv == thatv:
            continue


        a = (thisv[0], thisv[1])
        b = (thisv[0] + 2 * thisv[3], thisv[1] + 2 * thisv[4])
        c = (thatv[0], thatv[1])
        d = (thatv[0] + 2 * thatv[3], thatv[1] + 2 * thatv[4])

        intersection = line_intersection((a, b), (c, d))
        if intersection:
            #if 200000000000000 < intersection[0] < 400000000000000 and 200000000000000 < intersection[1] < 400000000000000:
            if SEARCH_MIN < intersection[0] < SEARCH_MAX and SEARCH_MIN < intersection[1] < SEARCH_MAX:
                crossing_hailstones.add((thisv, thatv))
                crossing_hailstones.add((thatv, thisv))
                crossing_hailstones2.add(thisv)
                crossing_hailstones2.add(thatv)
                #print(thisv, thatv)
                intersections.add(intersection)
                #intersections.append(intersection)
        
        checked.add((thisv,thatv))

#pprint.pprint(intersections)

print(len(crossing_hailstones) / 2)
print(len(crossing_hailstones2) / 2)
print(len(intersections))
print(intersections)



already checked ((19, 13, 30, -2, 1, -2), (18, 19, 22, -1, -1, -2))
already checked ((19, 13, 30, -2, 1, -2), (20, 25, 34, -2, -2, -4))
already checked ((18, 19, 22, -1, -1, -2), (20, 25, 34, -2, -2, -4))
already checked ((19, 13, 30, -2, 1, -2), (12, 31, 28, -1, -2, -1))
already checked ((18, 19, 22, -1, -1, -2), (12, 31, 28, -1, -2, -1))
already checked ((20, 25, 34, -2, -2, -4), (12, 31, 28, -1, -2, -1))
already checked ((19, 13, 30, -2, 1, -2), (20, 19, 15, 1, -5, -3))
already checked ((18, 19, 22, -1, -1, -2), (20, 19, 15, 1, -5, -3))
already checked ((20, 25, 34, -2, -2, -4), (20, 19, 15, 1, -5, -3))
already checked ((12, 31, 28, -1, -2, -1), (20, 19, 15, 1, -5, -3))
5.0
2.0
5
{(19.0, 24.0), (11.666666666666666, 16.666666666666668), (21.444444444444443, 11.777777777777779), (19.666666666666668, 20.666666666666668), (14.333333333333334, 15.333333333333334)}



200000000000000
284516192488866.9
400000000000000